In [4]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [5]:
NUM_RANDOM_SEEDS = 5
MODEL_CLASS = "lr"
EPSILON = 0.05
NUM_PERTURBED_MODELS = 100

In [7]:
data_source = "data/obermeyer/obermeyer_data_cleaned.csv"
prediction_output = "predictions/obermeyer/model_weight_perturbation_"+MODEL_CLASS+".csv"
target_variable = "cost_t"
features = ['dem_female', 'dem_age_band_18-24_tm1', 'dem_age_band_25-34_tm1', 'dem_age_band_35-44_tm1', 'dem_age_band_45-54_tm1',
            'dem_age_band_55-64_tm1', 'dem_age_band_65-74_tm1', 'dem_age_band_75+_tm1', 'hypertension_elixhauser_tm1', 'cost_dialysis_tm1',
            'cost_emergency_tm1', 'cost_home_health_tm1', 'cost_ip_medical_tm1', 'cost_ip_surgical_tm1', 'cost_laboratory_tm1',
            'cost_op_primary_care_tm1', 'cost_op_specialists_tm1', 'cost_op_surgery_tm1', 'cost_other_tm1', 'cost_pharmacy_tm1',
            'cost_physical_therapy_tm1', 'cost_radiology_tm1', 'gagne_sum_tm1']
other_variables = ['person_id', 'qualified_cost_25', 'qualified_cost_75', 'qualified_cost_50', 'qualified_gagne_1', 'qualified_gagne_2', 'qualified_gagne_3']

In [8]:
df = pd.read_csv(data_source)
X = df[features+other_variables]
y = df[target_variable]

In [9]:
def get_baseline_model_coefficients(X_train, y_train):
    if MODEL_CLASS == "lr":
        model = LinearRegression()
    print("beginning to train model")
    model.fit(X_train, y_train)
    print("model has been trained!")
    return model.coef_

In [10]:
def get_perturbed_weights(baseline_weights, epsilon):
    random_vector = np.random.randn(*baseline_weights.shape)
    random_vector = random_vector / np.linalg.norm(random_vector)
    perturbation = np.random.uniform(0, epsilon) * random_vector
    w = baseline_weights + perturbation
    if np.linalg.norm(w - baseline_weights) > epsilon:
        print(f"Oh no! Got a distance of {np.linalg.norm(w - baseline_weights)}")
    return w

In [11]:
def get_predictions_columns(X_test, baseline_weights, epsilon, y_true, baseline_loss):
    predictions = []
    columns = []
    count = 0
    for i in tqdm(range(NUM_PERTURBED_MODELS)):
        perturbed_weights = get_perturbed_weights(baseline_weights, epsilon)
        y_pred = np.dot(X_test, perturbed_weights)
        rss_loss = np.sum((y_true - y_pred) ** 2)
        if rss_loss - baseline_loss > epsilon:
            print(f"Uh oh! Got a RSS loss of {rss_loss}")
            count += 1
            continue
        if rss_loss < baseline_loss:
            print(f"perturbation {i} led to a better model")
        predictions.append(y_pred)
        columns.append(f'm_{i+1}')
    print(f"Number of models with high loss: {count}")
    return predictions, columns

In [ ]:
output = []
for random_seed in range(NUM_RANDOM_SEEDS):
    print("random seed", random_seed)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=random_seed)
    
    cost_threshold_25 = X_test['qualified_cost_25'].to_numpy()
    cost_threshold_50 = X_test['qualified_cost_50'].to_numpy()
    cost_threshold_75 = X_test['qualified_cost_75'].to_numpy()
    gagne_threshold_1 = X_test['qualified_gagne_1'].to_numpy()
    gagne_threshold_2 = X_test['qualified_gagne_2'].to_numpy()
    gagne_threshold_3 = X_test['qualified_gagne_3'].to_numpy()
    test_idx = X_test['person_id'].to_numpy()

    X_train = X_train.drop(columns=other_variables).to_numpy()
    print(X_train.shape)
    basis = sp.linalg.orth(X_train)
    basis = np.transpose(basis)
    print(basis.shape)
    X_train_proj = np.dot(X_train, basis)
    print(X_train_proj.shape)
    y_train = y_train.to_numpy()
    print(y_train.shape)
    X_test = X_test.drop(columns=other_variables).to_numpy()
    X_test_proj = np.dot(X_test, basis)
    y_test = y_test.to_numpy()
    print("arrays all set up")

    baseline_weights = get_baseline_model_coefficients(X_train_proj, y_train)
    baseline_prediction = np.dot(X_test_proj, baseline_weights)
    baseline_loss = np.sum((y_test - baseline_prediction) ** 2)
    print(f"baseline loss {baseline_loss}")
    predictions, columns = get_predictions_columns(X_test_proj, baseline_weights, EPSILON, y_test, baseline_loss)
    
    
    predictions_df = pd.DataFrame(predictions).transpose()
    predictions_df.columns=columns
    predictions_df["y"] = y_test
    predictions_df["person_id"] = test_idx 
    predictions_df["seed"] = random_seed
    
    predictions_df['qualified_cost_25'] = cost_threshold_25
    predictions_df['qualified_cost_50'] = cost_threshold_50
    predictions_df['qualified_cost_75'] = cost_threshold_75
    predictions_df['qualified_gagne_1'] = gagne_threshold_1
    predictions_df['qualified_gagne_2'] = gagne_threshold_2
    predictions_df['qualified_gagne_3'] = gagne_threshold_3

    output.append(predictions_df)
output = pd.concat(output)

random seed 0
(32685, 23)
(23, 32685)
(32685, 32685)
(32685,)
arrays all set up
beginning to train model


In [13]:
output.head()

,m_1,m_2,m_3,m_4,m_5,m_6,m_7,m_8,m_9,m_10,...,m_100,y,person_id,seed,qualified_cost_25,qualified_cost_50,qualified_cost_75,qualified_gagne_1,qualified_gagne_2,qualified_gagne_3
0,0.007204,0.007165,0.007269,0.007254,0.007408,0.007158,0.007330,0.007532,0.007118,0.007012,...,0.007425,0.009628,2545,0,0,1,1,1,0,0
1,0.010611,0.010546,0.010543,0.010884,0.010689,0.010488,0.010639,0.010649,0.010573,0.010342,...,0.010685,0.004905,8198,0,0,0,1,1,1,0
2,0.039960,0.039796,0.040069,0.040749,0.039703,0.039940,0.039863,0.039913,0.039872,0.040341,...,0.040129,0.009446,46461,0,0,1,1,0,0,0
3,-0.006376,-0.006321,-0.006375,-0.006416,-0.006561,-0.006431,-0.006416,-0.006416,-0.006390,-0.006561,...,-0.006479,0.002361,30620,0,0,0,1,0,0,0
4,-0.007962,-0.007938,-0.007759,-0.007795,-0.007665,-0.007974,-0.008205,-0.007668,-0.007907,-0.007779,...,-0.007599,0.003996,47418,0,0,0,1,1,1,0


In [14]:
output.info()

<class 'pandas.core.frame.DataFrame'>
Index: 80495 entries, 0 to 16098
Columns: 109 entries, m_1 to qualified_gagne_3
dtypes: float64(101), int64(8)
memory usage: 67.6 MB


In [17]:
output.to_csv(prediction_output, index=False)